In [ ]:
input_path = "."
output_path = "output"
relabel_output = '{}/{}'.format(output_path, 'needs_relabel')
approved_output = '{}/{}'.format(output_path, 'approved_labels')

In [ ]:
#Run this cell to look through pics after running all other cells.
next_button = widgets.Button(description="Approve labels")
reject_button = widgets.Button(description="Reject labels")

def approve_image(button_event):
    if len(prev_xml_files) > 0:
        approve_annotation(prev_xml_files.pop())
    display_next_image()
    
def reject_image(button_event):
    if len(prev_xml_files) > 0:
        reject_image_label(prev_xml_files.pop())
    display_next_image()
    
def display_next_image():
    if len(plt.get_fignums()) > 0:
        clear_output()
        plt.close('all')
    print('Number of images remaining: {}'.format(len(xml_files)))
    current_file = xml_files.pop()
    display_img('{}/{}'.format(input_path, current_file))
    prev_xml_files.append(current_file)
    
next_button.on_click(approve_image)
reject_button.on_click(reject_image)
display(next_button)
display(reject_button)

In [ ]:
from bs4 import BeautifulSoup
from IPython.display import display
from IPython.display import clear_output
from ipywidgets import widgets
import os
import shutil
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
from pylab import rcParams
rcParams['figure.figsize'] = (15, 15)
%matplotlib inline
#Lets us reference plots created in other cells. Aka make plots/figures global.
%config InlineBackend.close_figures=False

In [ ]:
def draw_annotation_rectangle(figure, labeled_object):
    bound_box = labeled_object.bndbox
    rectangle = figure.add_subplot(111, aspect='equal')
    xmin = int(bound_box.xmin.text)
    xmax = int(bound_box.xmax.text)
    ymin = int(bound_box.ymin.text)
    ymax = int(bound_box.ymax.text)
    rectangle.add_patch(patches.Rectangle((xmin, ymin), xmax-xmin, ymax-ymin, fill=False, 
                                          edgecolor=np.random.rand(3,1)))
    return labeled_object.find('name').text

def save_progress(xml_filename):
    with open('{}/progress.save'.format(input_path), 'a') as save_file:
        save_file.write(xml_filename + "\n")

def reject_image_label(xml_path):
    if not os.path.exists(relabel_output):
        os.makedirs(relabel_output)
    annotation = load_image_annotation('{}/{}'.format(input_path, xml_path))
    shutil.copyfile(annotation.path.text, '{}/{}.jpg'.format(relabel_output, annotation.filename.text))
    save_progress(xml_path)
    
def load_image_annotation(xml_filename):
    annotation = None
    with open(xml_filename) as xml_file:
        soup = BeautifulSoup(xml_file, 'lxml')
        annotation = soup.html.body.annotation
    return annotation

def display_img(xml_filename):
    annotation = load_image_annotation(xml_filename)
    img = mpimg.imread(annotation.path.text)
    figure = plt.figure()
    labels = []
    for index, labeled_object in enumerate(annotation.find_all('object')):
        labels.append(draw_annotation_rectangle(figure, labeled_object))
    plt.legend(labels=labels, loc="upper left", bbox_to_anchor=(1,1))
    plt.imshow(img, cmap='Set3')
    
def get_all_xml_files(directory):
    try:
        consumed_xml_files_newlined = list(open('{}/progress.save'.format(input_path), 'r'))
        consumed_xml_files = [filename.strip() for filename in consumed_xml_files_newlined]
    except:
        consumed_xml_files = []
    return [filename for filename in os.listdir(directory) 
            if ".xml" in filename and filename not in consumed_xml_files]

def approve_annotation(xml_filename):
    if not os.path.exists(approved_output):
        os.makedirs(approved_output)
    annotation = load_image_annotation('{}/{}'.format(input_path, xml_filename))
    shutil.copyfile(annotation.path.text, '{}/{}.jpg'.format(approved_output, annotation.filename.text))
    shutil.copyfile('{}/{}'.format(input_path, xml_filename), '{}/{}.jpg'.format(approved_output, xml_filename))
    save_progress(xml_filename)

In [ ]:
xml_files = get_all_xml_files(input_path)
prev_xml_files = []